In [ ]:
# Standard library
import os
from random import randint

# Third‑party: core
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Third‑party: scikit‑learn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    matthews_corrcoef as MCC,
    balanced_accuracy_score as BAS,
)

# Third‑party: TensorFlow / Keras (use tf.keras consistently)
import tensorflow as tf
from tensorflow.keras import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import (
    Input, Conv2D, SeparableConv2D, MaxPooling2D, Flatten,
    Dense, Dropout, BatchNormalization, GlobalAveragePooling2D,
    Concatenate,
)
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,
)
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.regularizers import l1, l2

# Reproducibility
print("TensorFlow Version:", tf.__version__)
tf.random.set_seed(42)


import os
import pandas as pd
from tqdm import tqdm

images = []
labels = []
# Specify the 'train' subfolder path
train_subfolder_path = os.path.join('/kaggle/input/alzheimers-dataset-4-class-of-images/Alzheimer_s Dataset', 'train')

# Loop through each subfolder in the 'train' directory
for folder in tqdm(os.listdir(train_subfolder_path)):
    subfolder_path = os.path.join(train_subfolder_path, folder)
    # Loop through each image file in the subfolder
    for image_filename in os.listdir(subfolder_path): 
        image_path = os.path.join(subfolder_path, image_filename)
        images.append(image_path)
        labels.append(folder)  # The label is the name of the subfolder

df = pd.DataFrame({'image': images, 'label': labels})
print(df)


print(df.groupby(['label']).count())

plt.figure(figsize=(15,8))
ax = sns.countplot(x=df.label,palette='Set1')
ax.set_xlabel("Class",fontsize=20)
ax.set_ylabel("Count",fontsize=20)
plt.title('The Number Of Samples For Each Class',fontsize=20)
plt.grid(True)
plt.xticks(rotation=45)
plt.show()


In [ ]:
class_names = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]


In [ ]:
class_names = ["MD", "MoD", "ND","VMildD"]
# ["Mild AD", "Moderate AD", "Non AD", "Very Mild AD"]
print(class_names)

# **Processing Kaggle dataset of 4 AD classes containing 6400 images**

import os
folder_path = '/kaggle/working/output/'
# Create the new folder
os.mkdir(folder_path)

#output_dir = pathlib.Path('/kaggle/input/dataset-alzheimer/Alzheimer_s Dataset/train')
output_dir = pathlib.Path('/kaggle/input/alzheimer-mri-4-classes-dataset/Alzheimer_MRI_4_classes_dataset')
#output_dir = pathlib.Path('/kaggle/input/alzheimer-mri-ds/Alzheimer_MRI_ds/Train')
image_count_train = len(list(output_dir.glob('*/*.jpg')))
print(image_count_train)

import shutil
directory_path = "/kaggle/working/sample_dataset"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)
else:
    print(f"The directory {directory_path} does not exist.")


# Define the directories for saving the split data
train_dir = 'train'
test_dir = 'test'
class_names = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']

# Create the class-labeled folders in both train and test directories
for class_name in class_names:
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

# Function to save images into respective class folders
def save_images(data, labels, folder_dir):
    for i, image in enumerate(data):
        label_idx = np.argmax(labels[i])  # Get the class index
        class_name = class_names[label_idx]
        image_path = os.path.join(folder_dir, class_name, f'image_{i}.png')
        # Convert image back to [0,255] and save
        image = (image * 255).astype(np.uint8)
        tf.keras.preprocessing.image.save_img(image_path, image)

# Save training images
save_images(X_train_data, y_train_labels, train_dir)

# Save testing images
save_images(X_test, y_test, test_dir)

print(f"Training images saved to {train_dir}")
print(f"Testing images saved to {test_dir}")

**resizing the saved images**

In [ ]:
output_dir_train='/kaggle/working/train'
from tensorflow.keras.preprocessing.image import ImageDataGenerator
Size=(128, 128)#(176, 208)#(200,200)#(176,176)#(224, 224)
work_dr = ImageDataGenerator(
    rescale = 1./255
)
train_data_gen = work_dr.flow_from_directory(output_dir_train,target_size=Size, batch_size=5120, shuffle=True)

for i in range(len(train_data_gen)):
    X_train_data, y_train_labels = train_data_gen[i]
print(X_train_data.shape, y_train_labels.shape) 

In [ ]:
output_dir_test='/kaggle/working/test'
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Size=(150, 150)
Size=(128, 128)#(176, 208)#(200,200)#(176,176)#(224, 224)
work_dr = ImageDataGenerator(
    rescale = 1./255
)
test_data_gen = work_dr.flow_from_directory(output_dir_test,target_size=Size, batch_size=1280,shuffle=True)

for i in range(len(test_data_gen)):
    X_test, y_test = test_data_gen[i]
print( X_test.shape, y_test.shape) 

In [ ]:
# Define a function to display images
def display_loaded_images(images, labels, class_indices, num_images=9):
    label_names = {v: k for k, v in class_indices.items()}  # Map label indices to class names
    plt.figure(figsize=(10, 10))
    
    for i in range(num_images):
        plt.subplot(3, 3, i + 1)  # Create a 3x3 grid
        plt.imshow(images[i])  # Display the image
        label_idx = np.argmax(labels[i])  # Get the label index
        plt.title(label_names[label_idx])  # Display the class name as the title
        plt.axis('off')  # Hide the axes for a cleaner look

    plt.show()

# Display the first 9 images and their labels
print("Displaying training images:")
display_loaded_images(X_train_data, y_train_labels, train_data_gen.class_indices)

**Proposed Dual branch 5 CNN layered model architecture**

In [ ]:
    image_width=128 
    image_height=128 
    
    input_shape=(image_width, image_height, 3)
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(16, (3,3), activation='relu', kernel_regularizer=l2(0.02))(inputs)
    bn1_1a = BatchNormalization()(conv1)
    conv1a = Conv2D(32, (3,3), activation='relu',kernel_regularizer=l2(0.02))(bn1_1a)
    bn1_1 = BatchNormalization()(conv1a)
    pool1_1 = MaxPooling2D()(bn1_1)
    conv2a = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool1_1)
    bn1_2 = BatchNormalization()(conv2a)
    pool1_2 = MaxPooling2D()(bn1_2)
    conv3 = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool1_2)
    bn1_3 = BatchNormalization()(conv3)
    pool1_3 = MaxPooling2D()(bn1_3)
    conv4 = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool1_3)
    bn1_4 = BatchNormalization()(conv4)
    pool1_4 = MaxPooling2D()(bn1_4)
    x1 = Flatten()(pool1_4)
    x1 = Dense(64, activation='relu')(x1)
 
    conv2_1 = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(inputs)
    bn2_1a = BatchNormalization()(conv2_1)
    conv2_1a = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(bn2_1a)
    bn2_1 = BatchNormalization()(conv2_1a)
    pool2_1 = MaxPooling2D()(bn2_1)
 
    conv2_2a =  Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool2_1)
    bn2_2 = BatchNormalization()(conv2_2a)
    pool2_2 = MaxPooling2D()(bn2_2)
     
    conv2_3 =  Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(bn2_2)
    bn2_3 = BatchNormalization()(conv2_3)
    pool2_3 = MaxPooling2D()(bn2_3)
    
    conv2_4 =  Conv2D(16, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool2_3)
    bn2_4 = BatchNormalization()(conv2_4)
    pool2_4 = MaxPooling2D()(bn2_4)

    x2 = Flatten()(pool2_4)
    x2 = Dense(64, activation='relu')(x2)
    
    x = Concatenate()([x1,x2])
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    output = Dense(4, activation='softmax')(x)  # 4 classes for classification
    custom_model = Model(inputs=inputs, outputs=output, name="CNNs_AD4_5cnnL")

    METRICS = [
        tf.keras.metrics.CategoricalAccuracy(name='acc'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.F1Score(name='f1_score')]


   

    custom_model.compile(optimizer=Adam(learning_rate=0.0005),
                              loss=tf.losses.CategoricalCrossentropy(),
                              metrics=METRICS)

    custom_model.summary()





In [ ]:
    checkpointest = tf.keras.callbacks.ModelCheckpoint(
    filepath='/kaggle/working/final_AD4_CNNs_5L.keras',
    save_weights_only=False,
    monitor='val_acc',  # Monitor the categorical accuracy for multiclass classification
    save_best_only=True,
    save_freq="epoch"
    )

    # Define custom callback to stop training when accuracy exceeds 99%
    class MyCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('acc') > 0.999:  # Use 'categorical_accuracy' for accuracy metric
                print("\nReached accuracy threshold! Terminating training.")
                self.model.stop_training = True

    # Instantiate custom callback
    my_callbacktest = MyCallback()

    # Define EarlyStopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )
history = custom_model.fit(X_train_data, y_train_labels, 
                                validation_split=0.2, 
                                epochs=30, batch_size=32, 
                                callbacks=[checkpointest, my_callbacktest, early_stopping], 
                                shuffle=True)

**K-fold cross validation**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
import numpy as np

checkpointest = tf.keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/final_AD4_CNNs_5cnnL_10f.keras',
    save_weights_only=False,
    monitor='val_acc',  # Monitor the categorical accuracy for multiclass classification
    save_best_only=True,
    save_freq="epoch"
    )

    # Define custom callback to stop training when accuracy exceeds 99%
class MyCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('acc') > 0.999:  # Use 'categorical_accuracy' for accuracy metric
                print("\nReached accuracy threshold! Terminating training.")
                self.model.stop_training = True

    # Instantiate custom callback
my_callbacktest = MyCallback()

    # Define EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True
    )
#histor

    # Define learning rate scheduler callback
def lr_scheduler(epoch, lr, epochs=60):
        initial = 1e-3
        if epoch <= epochs * 0.1:
            return initial * 1.0
        elif epoch > epochs * 0.1 and epoch < epochs * 0.25:
            new_lr = lr * tf.math.exp(-0.1).numpy()
            print("Learning rate after exponential decay:", new_lr)
            return new_lr
        else:
            new_lr = lr * tf.math.exp(-0.008).numpy()
            print("Learning rate after exp_decay:", new_lr)
            return new_lr

lr_scheduling = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
    
# Initialize StratifiedKFold # keep n_splits=5 for 5 fold and n_splits=8 for 8-fold
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Lists to store results
X_train_list, X_val_list, y_train_list, y_val_list = [], [], [], []
history_list = []
fold = 0 

# Perform StratifiedKFold split and store train/validation data
for train_index, val_index in skf.split(X_train_data, np.argmax(y_train_labels, axis=1)):
    X_train_fold, X_val_fold = X_train_data[train_index], X_train_data[val_index]
    y_train_fold, y_val_fold = y_train_labels[train_index], y_train_labels[val_index]
    
    X_train_list.append(X_train_fold)
    X_val_list.append(X_val_fold)
    y_train_list.append(y_train_fold)
    y_val_list.append(y_val_fold)

# Print shapes of the datasets
for i in range(len(X_train_list)):
    print('Fold', i+1)
    print('X_train shape:', X_train_list[i].shape)
    print('y_train shape:', y_train_list[i].shape)
    print('X_val shape:', X_val_list[i].shape)
    print('y_val shape:', y_val_list[i].shape)
    
    # Train the model
    history = custom_model.fit(X_train_fold, y_train_fold, 
                                validation_data=(X_val_fold, y_val_fold), 
                                epochs=30, batch_size=32, 
                                callbacks=[checkpointest, my_callbacktest, early_stopping], 
                                shuffle=True)
    
    # Save training history for this fold
    history_list.append(history.history)
    import pickle
    # Save the training histories for all folds
    with open('/kaggle/working/CNNs_training_AD4_10f_final.pkl', 'wb') as file:
        pickle.dump(history_list, file)
    # Save the trained model
    custom_model.save('/kaggle/working/final_CNNs_AD4_model_10fold{}.keras'.format(fold))
    
    fold += 1
    
import pickle
# Save the training histories for all folds
with open('/kaggle/working/CNNs_training_AD4_10f_final.pkl', 'wb') as file:
    pickle.dump(history_list, file)

In [ ]:
import pickle
with open('/kaggle/working/CNNs_training_AD4_10f_final.pkl', 'rb') as file:

    history_list = pickle.load(file)

In [ ]:
import matplotlib.pyplot as plt

# Iterate over each fold's training history
for i, history in enumerate(history_list):
    plt.figure(figsize=(10, 5))
    
    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history['acc'],'bo--',linewidth=2)
    plt.plot(history['val_acc'],'ro--',linewidth=2)
    plt.title('Model accuracy - Fold {}'.format(i+1))
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.xticks(range(0,len(history['acc'])+1,2), range(0, len(history['acc'])+1,2))  # Set x-axis ticks with interval of 5 epochs
    plt.ylim(0, 1)  # Set y-axis limits from 0 to 1
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history['loss'],'bo--',linewidth=2)
    plt.plot(history['val_loss'],'ro--',linewidth=2)
    plt.title('Model loss - Fold {}'.format(i+1))
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.xticks(range(0,len(history['loss'])+1,2), range(0, len(history['loss'])+1,2) ) # Force integer ticks
    #plt.ylim(0, 1)  # Set y-axis limits from 0 to 1
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.tight_layout()
    plt.show()


**8-fold training history**

In [ ]:
import matplotlib.pyplot as plt
import pickle
with open('/kaggle/working/CNNs_training_AD4_8f_final.pkl', 'rb') as file:

    history_list = pickle.load(file)
# Iterate over each fold's training history
for i, history in enumerate(history_list):
    plt.figure(figsize=(10, 5))
    
    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history['acc'],'bo--',linewidth=2)
    plt.plot(history['val_acc'],'ro--',linewidth=2)
    plt.title('Model accuracy - Fold {}'.format(i+1))
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.xticks(range(0,len(history['acc'])+1,2), range(0, len(history['acc'])+1,2))  # Set x-axis ticks with interval of 5 epochs
    plt.ylim(0, 1)  # Set y-axis limits from 0 to 1
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history['loss'],'bo--',linewidth=2)
    plt.plot(history['val_loss'],'ro--',linewidth=2)
    plt.title('Model loss - Fold {}'.format(i+1))
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.xticks(range(0,len(history['loss'])+1,2), range(0, len(history['loss'])+1,2) ) # Force integer ticks
    #plt.ylim(0, 1)  # Set y-axis limits from 0 to 1
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    plt.tight_layout()
    plt.show()


In [ ]:
model8f=tf.keras.models.load_model("/kaggle/working/final_AD4_CNNs_5cnnL_8f.keras")
test_scores = model8f.evaluate(X_test, y_test)
print("Testing Accuracy: %.4f%%" % (test_scores[1] * 100))
#Print the classification report of the tested data
pred_labels = model8f.predict(X_test)
#Since the labels are softmax arrays, we need to roundoff to have it in the form of 0s and 1s,
#similar to the test_labels
def roundoff(arr):
    """To round off according to the argmax of each predicted label array. """
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

for labels in pred_labels:
    labels = roundoff(labels)

# Classification report
#print(classification_report(np.argmax(y_test, axis=1), np.argmax(pred_labels, axis=1)))
print(classification_report(y_test, pred_labels, target_names=class_names))


# Plot the confusion matrix to understand the classification in detailsnio
pred_ls = np.argmax(pred_labels, axis=1)
test_ls = np.argmax(y_test, axis=1)

conf_arr = confusion_matrix(test_ls, pred_ls)

plt.figure(figsize=(16, 12), dpi=200, facecolor='w', edgecolor='k')

#ax = sns.heatmap(conf_arr, cmap='Greens', annot=True, fmt='d', xticklabels=class_names,yticklabels=class_names, annot_kws={"fontsize": 24})
 #Plot heatmap
ax = sns.heatmap(conf_arr, cmap='Blues', annot=True, fmt='d')
# Increase font size for annotations
for text in ax.texts:
    text.set_fontsize(36)  # Adjust the font size as needed

# Increase font size for x-axis and y-axis labels
#plt.xticks(np.arange(len(class_names)), class_names, fontsize=14)
#plt.yticks(np.arange(len(class_names)), class_names, fontsize=14)

ax.set_xticklabels(class_names, rotation=0, ha='center', fontsize=24)
ax.set_yticklabels(class_names, rotation=0, va='center', fontsize=24)

#plt.title("AD classification, Batch Size=32, 8fold", fontsize=16)  # Title font size
plt.xlabel("Prediction", fontsize=18)  # X label font size
plt.ylabel("Actual", fontsize=18)  # Y label font size
plt.show(ax)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
class_names=["MD", "MoD", "ND", "VMD"]
# 1) Predict and convert to class ids
pred_probs = model8f.predict(X_test, verbose=0)
y_true = np.argmax(y_test, axis=1) if y_test.ndim == 2 else y_test
y_pred = np.argmax(pred_probs, axis=1)

# 2) Classification report as a DataFrame
report_dict = classification_report(
    y_true,
    y_pred,
    target_names=class_names,
    output_dict=True,
    digits=4
)
report_df = pd.DataFrame(report_dict).T

# Keep per-class rows + macro/weighted averages; drop 'support' from heatmap
rows = list(class_names) + ['weighted avg']
report_heat = report_df.loc[rows, ['precision', 'recall', 'f1-score']]

# (optional) show class supports in the y-axis labels
supports = np.bincount(y_true, minlength=len(class_names))
row_labels = [f'{c} (n={s})' for c, s in zip(class_names, supports)] + ['Average']
report_heat.index = row_labels

# 3) Plot the classification report heatmap
plt.figure(figsize=(8, 0.6*len(report_heat)+2), dpi=200)
ax = sns.heatmap(
    report_heat, annot=True, fmt=".4f", cmap="coolwarm", cbar=True, linewidths=0.5, annot_kws={"size": 20} )
#ax = sns.heatmap(report_heat, annot=True, fmt='.3f', cmap='Greens', vmin=0, vmax=1, cbar=True)
ax.set_title('Classification Report (heatmap)')
ax.set_xlabel('Metric')
ax.set_ylabel('Class')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, va='center')
plt.tight_layout()
plt.show()

# 4) (optional) Confusion matrix heatmap (you already have this)
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6), dpi=200)
ax = sns.heatmap(conf_arr, cmap='Blues', annot=True, fmt='d')
# Increase font size for annotations
for text in ax.texts:
    text.set_fontsize(24)  # Adjust the font size as needed

# Increase font size for x-axis and y-axis labels
#plt.xticks(np.arange(len(class_names)), class_names, fontsize=14)
#plt.yticks(np.arange(len(class_names)), class_names, fontsize=14)

ax.set_xticklabels(class_names, rotation=0, ha='center', fontsize=20)
ax.set_yticklabels(class_names, rotation=0, va='center', fontsize=20)

#plt.title("AD classification, Batch Size=32, 8fold", fontsize=16)  # Title font size
plt.xlabel("Prediction", fontsize=14)  # X label font size
plt.ylabel("Actual", fontsize=14)  # Y label font size
plt.show(ax)


In [ ]:
#5-fold
model5f=tf.keras.models.load_model('/kaggle/working/final_AD4_CNNs_5cnnL.keras')
test_scores = model5f.evaluate(X_test, y_test)
print("Testing Accuracy: %.4f%%" % (test_scores[1] * 100))

#Print the classification report of the tested data
pred_labels = model5f.predict(X_test)
#Since the labels are softmax arrays, we need to roundoff to have it in the form of 0s and 1s,
#similar to the test_labels
def roundoff(arr):
    """To round off according to the argmax of each predicted label array. """
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

for labels in pred_labels:
    labels = roundoff(labels)

# Classification report
#print(classification_report(np.argmax(y_test, axis=1), np.argmax(pred_labels, axis=1)))
print(classification_report(y_test, pred_labels, target_names=class_names))


# Plot the confusion matrix to understand the classification in detailsnio
pred_ls = np.argmax(pred_labels, axis=1)
test_ls = np.argmax(y_test, axis=1)

conf_arr = confusion_matrix(test_ls, pred_ls)

plt.figure(figsize=(16, 12), dpi=200, facecolor='w', edgecolor='k')

#ax = sns.heatmap(conf_arr, cmap='Greens', annot=True, fmt='d', xticklabels=class_names,yticklabels=class_names, annot_kws={"fontsize": 24})
 #Plot heatmap
ax = sns.heatmap(conf_arr, cmap='Blues', annot=True, fmt='d')
# Increase font size for annotations
for text in ax.texts:
    text.set_fontsize(36)  # Adjust the font size as needed

# Increase font size for x-axis and y-axis labels
#plt.xticks(np.arange(len(class_names)), class_names, fontsize=14)
#plt.yticks(np.arange(len(class_names)), class_names, fontsize=14)

ax.set_xticklabels(class_names, rotation=0, ha='center', fontsize=24)
ax.set_yticklabels(class_names, rotation=0, va='center', fontsize=24)

#plt.title("AD classification, Batch Size=32, 5fold", fontsize=16)  # Title font size
plt.xlabel("Prediction", fontsize=18)  # X label font size
plt.ylabel("Actual", fontsize=18)  # Y label font size
plt.show(ax)

In [ ]:
#10-fold
model10f=tf.keras.models.load_model('/kaggle/working/final_AD4_CNNs_5cnnL_10f.keras')
test_scores = model10f.evaluate(X_test, y_test)
print("Testing Accuracy: %.4f%%" % (test_scores[1] * 100))

#Print the classification report of the tested data
pred_labels = model10f.predict(X_test)
#Since the labels are softmax arrays, we need to roundoff to have it in the form of 0s and 1s,
#similar to the test_labels
def roundoff(arr):
    """To round off according to the argmax of each predicted label array. """
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

for labels in pred_labels:
    labels = roundoff(labels)

# Classification report
#print(classification_report(np.argmax(y_test, axis=1), np.argmax(pred_labels, axis=1)))
print(classification_report(y_test, pred_labels, target_names=class_names))


# Plot the confusion matrix to understand the classification in detailsnio
pred_ls = np.argmax(pred_labels, axis=1)
test_ls = np.argmax(y_test, axis=1)

conf_arr = confusion_matrix(test_ls, pred_ls)

plt.figure(figsize=(16, 12), dpi=200, facecolor='w', edgecolor='k')

#ax = sns.heatmap(conf_arr, cmap='Greens', annot=True, fmt='d', xticklabels=class_names,yticklabels=class_names, annot_kws={"fontsize": 24})
 #Plot heatmap
ax = sns.heatmap(conf_arr, cmap='Blues', annot=True, fmt='d')
# Increase font size for annotations
for text in ax.texts:
    text.set_fontsize(36)  # Adjust the font size as needed

# Increase font size for x-axis and y-axis labels
#plt.xticks(np.arange(len(class_names)), class_names, fontsize=14)
#plt.yticks(np.arange(len(class_names)), class_names, fontsize=14)

ax.set_xticklabels(class_names, rotation=0, ha='center', fontsize=24)
ax.set_yticklabels(class_names, rotation=0, va='center', fontsize=24)

#plt.title("AD classification, Batch Size=32, 5fold", fontsize=16)  # Title font size
plt.xlabel("Prediction", fontsize=18)  # X label font size
plt.ylabel("Actual", fontsize=18)  # Y label font size
plt.show(ax)

In [ ]:
    from keras.optimizers import RMSprop
    from tensorflow.keras.regularizers import l1_l2

    image_width=128 
    image_height=128 
    input_shape=(image_width, image_height, 3)
 
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(16, (3,3), activation='relu', kernel_regularizer=l2(0.02))(inputs)
    bn1_1a = BatchNormalization()(conv1)
    
    conv1a = Conv2D(32, (3,3), activation='relu',kernel_regularizer=l2(0.02))(bn1_1a)
    bn1_1 = BatchNormalization()(conv1a)
    #pool1_1 = MaxPooling2D()(bn1_1)
    
    conv2a = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool1_1)
    bn1_2 = BatchNormalization()(conv2a)
    pool1_2 = MaxPooling2D()(bn1_2)
    conv3 = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool1_2)
    bn1_3 = BatchNormalization()(conv3)
    pool1_3 = MaxPooling2D()(bn1_3)
 
    conv4 = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool1_3)
    bn1_4 = BatchNormalization()(conv4)
    pool1_4 = MaxPooling2D()(bn1_4)

    x1 = Flatten()(pool1_4)
    x1 = Dense(64, activation='relu')(x1)
 
    conv2_1 = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(inputs)
    bn2_1a = BatchNormalization()(conv2_1)
    conv2_1a = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(bn2_1a)
    bn2_1 = BatchNormalization()(conv2_1a)
    pool2_1 = MaxPooling2D()(bn2_1)
 
    conv2_2a =  Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool2_1)
    bn2_2 = BatchNormalization()(conv2_2a)
    pool2_2 = MaxPooling2D()(bn2_2)
     
    conv2_3 =  Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool2_2)
    bn2_3 = BatchNormalization()(conv2_3)
    pool2_3 = MaxPooling2D()(bn2_3)
    
    conv2_4 =  Conv2D(16, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool2_3)
    bn2_4 = BatchNormalization()(conv2_4)
    pool2_4 = MaxPooling2D()(bn2_4)

    x2 = Flatten()(pool2_4)
    x2 = Dense(64, activation='relu')(x2)
    
    x = Concatenate()([x1,x2])

    x = Dropout(0.4)(x)
    x = Dense(512, activation='relu')(x)
 

    output = Dense(4, activation='softmax')(x)  # Assuming 5 classes for classification

    METRICS = [
        tf.keras.metrics.CategoricalAccuracy(name='acc'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.F1Score(name='f1_score')]


    custom_model2 = Model(inputs=inputs, outputs=output, name="CNNs_AD4_128_3cnnL")

    custom_model2.compile(optimizer=Adam(learning_rate=0.0005),
                              loss=tf.losses.CategoricalCrossentropy(),
                              metrics=METRICS)

    custom_model2.summary()





In [ ]:
#model with 3 CNN layers
# Evaluating the model on the data
#test_scores = custom_model2.evaluate(X_test, y_test)
#print("Testing Accuracy: %.4f%%" % (test_scores[1] * 100))
model=tf.keras.models.load_model('/kaggle/working/test_AD4_CNNs_3L.keras')
test_scores = model.evaluate(X_test, y_test)
print("Testing Accuracy: %.4f%%" % (test_scores[1] * 100))

In [ ]:
# Evaluating the model on the data
#test_scores = custom_model2.evaluate(X_test, y_test)
#print("Testing Accuracy: %.4f%%" % (test_scores[1] * 100))
#model with 4 CNN layers
model=tf.keras.models.load_model('/kaggle/working/test_AD4_CNNs_4L.keras')
test_scores = model.evaluate(X_test, y_test)
print("Testing Accuracy: %.4f%%" % (test_scores[1] * 100))

In [ ]:
X_test=test_data
y_test=test_labels

# **OASIS**

In [ ]:
import os, random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# --- 1) Full determinism ---
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
try:
    tf.config.experimental.enable_op_determinism(True)
except Exception:
    pass

# --- 2) Load data ONCE in a fixed order (no shuffle) ---
output_dir = '/kaggle/working/sample_dataset' # saved the images in the working directory from input
Size = (64, 64)

datagen = ImageDataGenerator(rescale=1./255)
gen = datagen.flow_from_directory(
    output_dir,
    target_size=Size,
    batch_size=99999,          # large to fetch all in one batch
    shuffle=False,              # <- critical for reproducibility
    class_mode='categorical'    # one-hot labels to match your model
)

X_all, Y_all = next(gen)        # X_all: (N,H,W,C), Y_all: (N,4) one-hot
y_ids = np.argmax(Y_all, axis=1)  # class ids for stratification only
N = X_all.shape[0]
idx = np.arange(N)

# --- 3) Deterministic stratified split BY INDICES ---
idx_train, idx_test = train_test_split(
    idx,
    test_size=0.2,
    random_state=SEED,
    shuffle=True,
    stratify=y_ids
)

X_train_data = X_all[idx_train]
X_test        = X_all[idx_test]
y_train_labels = Y_all[idx_train]   # KEEP one-hot for your categorical loss
y_test         = Y_all[idx_test]    # KEEP one-hot for evaluation

print('X_train', X_train_data.shape)
print('X_test',  X_test.shape)
print('y_train', y_train_labels.shape)
print('y_test',  y_test.shape)




In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
import numpy as np

checkpointest = tf.keras.callbacks.ModelCheckpoint(filepath='/kaggle/working/model64_AD4_OAsis_5cnnl.keras',
    save_weights_only=False,
    monitor='val_acc',  # Monitor the categorical accuracy for multiclass classification
    save_best_only=True,
    save_freq="epoch"
    )

    # Define custom callback to stop training when accuracy exceeds 99%
class MyCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('acc') > 0.999:  # Use 'categorical_accuracy' for accuracy metric
                print("\nReached accuracy threshold! Terminating training.")
                self.model.stop_training = True

    # Instantiate custom callback
my_callbacktest = MyCallback()

    # Define EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True
    )
#histor

    # Define learning rate scheduler callback
def lr_scheduler(epoch, lr, epochs=60):
        initial = 1e-3
        if epoch <= epochs * 0.1:
            return initial * 1.0
        elif epoch > epochs * 0.1 and epoch < epochs * 0.25:
            new_lr = lr * tf.math.exp(-0.1).numpy()
            print("Learning rate after exponential decay:", new_lr)
            return new_lr
        else:
            new_lr = lr * tf.math.exp(-0.008).numpy()
            print("Learning rate after exp_decay:", new_lr)
            return new_lr

lr_scheduling = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [ ]:
    from keras.optimizers import RMSprop
    from tensorflow.keras.regularizers import l1_l2
    from tensorflow.keras import Sequential, Input
    from tensorflow.keras.layers import Dense, Dropout
    from tensorflow.keras.layers import Conv2D, Flatten
    import pathlib
    import tensorflow as tf
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import os
    import PIL
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.models import Sequential
    from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
    from tensorflow.keras.optimizers import Adam # - Works
    import random
    from glob import glob
    import seaborn as sns
    from tensorflow.keras.losses import SparseCategoricalCrossentropy
    import matplotlib.pyplot as plt
    import matplotlib.image as img
    import warnings
    warnings.filterwarnings('ignore')
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
    image_width=64#128 
    image_height=64#128 
    input_shape=(image_width, image_height, 3)
 
    inputs = Input(shape=input_shape)
    conv1 = Conv2D(16, (3,3), activation='relu', kernel_regularizer=l2(0.02))(inputs)
    bn1_1a = BatchNormalization()(conv1)
    conv1a = Conv2D(32, (3,3), activation='relu',kernel_regularizer=l2(0.02))(bn1_1a)
    bn1_1 = BatchNormalization()(conv1a)
    pool1_1 = MaxPooling2D()(bn1_1)
    conv2a = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool1_1)
    bn1_2 = BatchNormalization()(conv2a)
    pool1_2 = MaxPooling2D()(bn1_2)
    conv3 = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool1_2)
    bn1_3 = BatchNormalization()(conv3)
    pool1_3 = MaxPooling2D()(bn1_3)
    conv4 = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool1_3)
    bn1_4 = BatchNormalization()(conv4)
    pool1_4 = MaxPooling2D()(bn1_4)
    x1 = Flatten()(pool1_4)
    x1 = Dense(64, activation='relu')(x1)
 
    conv2_1 = Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(inputs)
    bn2_1a = BatchNormalization()(conv2_1)
    conv2_1a = Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(bn2_1a)
    bn2_1 = BatchNormalization()(conv2_1a)
    pool2_1 = MaxPooling2D()(bn2_1)
 
    conv2_2a =  Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool2_1)
    bn2_2 = BatchNormalization()(conv2_2a)
    pool2_2 = MaxPooling2D()(bn2_2)
     
    conv2_3 =  Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(bn2_2)
    bn2_3 = BatchNormalization()(conv2_3)
    pool2_3 = MaxPooling2D()(bn2_3)
    
    conv2_4 =  Conv2D(16, (3, 3), activation='relu', kernel_regularizer=l2(0.02))(pool2_3)
    bn2_4 = BatchNormalization()(conv2_4)
    pool2_4 = MaxPooling2D()(bn2_4)

    x2 = Flatten()(pool2_4)
    x2 = Dense(64, activation='relu')(x2)
    
    x = Concatenate()([x1,x2])
    x = Dropout(0.2)(x)
    x = Dense(512, activation='relu')(x)
    

    output = Dense(4, activation='softmax')(x)  # Assuming 5 classes for classification

    METRICS = [
        tf.keras.metrics.CategoricalAccuracy(name='acc'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.AUC(name='auc'),
        tf.keras.metrics.F1Score(name='f1_score')]


    model64 = Model(inputs=inputs, outputs=output, name="CNNs_AD4_64_5cnnL_Oasis")

    model64.compile(optimizer=Adam(learning_rate=0.0005),
                              loss=tf.losses.CategoricalCrossentropy(),
                              metrics=METRICS)

    model64.summary()



In [ ]:
history = model64.fit(X_train_data, y_train_labels, validation_split=0.1, epochs=30,batch_size=32,
                               callbacks=[checkpointest, my_callbacktest, early_stopping], 
                                shuffle=True)# early_stopping,lr_scheduling, 
import pickle

    # Assuming 'history' is the variable containing the training history
with open('/kaggle/working/train_oasis_cnn5l.pkl', 'wb') as file:
        pickle.dump(history.history, file)

In [ ]:
# Plotting the trend of the metrics during training
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 3, figsize=(30, 6))
ax = ax.ravel()

for i, metric in enumerate(["acc", "auc", "loss"]):
    ax[i].plot(history.history[metric],'bo--',linewidth=2)
    ax[i].plot(history.history["val_" + metric], "ro--",linewidth=2)
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("Epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["Training", "Validation"])

#plt.ylim(0, 1)  # Set y-axis limits from 0 to 1

In [ ]:
model64=tf.keras.models.load_model("/kaggle/working/model64_AD4_OAsis_5cnnl.keras")
test_scores = model64.evaluate(X_test, y_test)
print("Testing Accuracy: %.2f%%" % (test_scores[1] * 100))


In [ ]:
model64=tf.keras.models.load_model("/kaggle/working/model64_AD4_OAsis.keras")
test_scores = model64.evaluate(X_test, y_test)
print("Testing Accuracy: %.2f%%" % (test_scores[1] * 100))

In [ ]:
test_scores = model64.evaluate(X_test, y_test)
print("Testing Accuracy: %.2f%%" % (test_scores[1] * 100))

In [ ]:
#Print the classification report of the tested data
pred_labels = model64.predict(X_test)
#Since the labels are softmax arrays, we need to roundoff to have it in the form of 0s and 1s,
#similar to the test_labels
def roundoff(arr):
    """To round off according to the argmax of each predicted label array. """
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

for labels in pred_labels:
    labels = roundoff(labels)

# Classification report
#print(classification_report(np.argmax(y_test, axis=1), np.argmax(pred_labels, axis=1)))
print(classification_report(y_test, pred_labels, target_names=class_names, digits=4))


# Plot the confusion matrix to understand the classification in detailsnio
pred_ls = np.argmax(pred_labels, axis=1)
test_ls = np.argmax(y_test, axis=1)

conf_arr = confusion_matrix(test_ls, pred_ls)

plt.figure(figsize=(16, 12), dpi=200, facecolor='w', edgecolor='k')

#ax = sns.heatmap(conf_arr, cmap='Greens', annot=True, fmt='d', xticklabels=class_names,yticklabels=class_names, annot_kws={"fontsize": 24})
 #Plot heatmap
ax = sns.heatmap(conf_arr, cmap='Greens', annot=True, fmt='d')
# Increase font size for annotations
for text in ax.texts:
    text.set_fontsize(36)  # Adjust the font size as needed

# Increase font size for x-axis and y-axis labels
#plt.xticks(np.arange(len(class_names)), class_names, fontsize=14)
#plt.yticks(np.arange(len(class_names)), class_names, fontsize=14)

ax.set_xticklabels(class_names, rotation=45, ha='right', fontsize=24)
ax.set_yticklabels(class_names, rotation=0, va='center', fontsize=24)

plt.title(" 64x64 image size", fontsize=16)  # Title font size
plt.xlabel("Prediction", fontsize=18)  # X label font size
plt.ylabel("Truth", fontsize=18)  # Y label font size
plt.show(ax)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
class_names=["MD", "MoD", "ND", "VMD"]
# 1) Predict and convert to class ids
pred_probs = model64.predict(X_test, verbose=0)
y_true = np.argmax(y_test, axis=1) if y_test.ndim == 2 else y_test
y_pred = np.argmax(pred_probs, axis=1)

# 2) Classification report as a DataFrame
report_dict = classification_report(
    y_true,
    y_pred,
    target_names=class_names,
    output_dict=True,
    digits=4
)
report_df = pd.DataFrame(report_dict).T

# Keep per-class rows + macro/weighted averages; drop 'support' from heatmap
rows = list(class_names) + ['weighted avg']
report_heat = report_df.loc[rows, ['precision', 'recall', 'f1-score']]

# (optional) show class supports in the y-axis labels
supports = np.bincount(y_true, minlength=len(class_names))
row_labels = [f'{c} (n={s})' for c, s in zip(class_names, supports)] + ['Average']
report_heat.index = row_labels

# 3) Plot the classification report heatmap
plt.figure(figsize=(8, 0.6*len(report_heat)+2), dpi=200)
ax = sns.heatmap(
    report_heat, annot=True, fmt=".4f", cmap="coolwarm", cbar=True, linewidths=0.5, annot_kws={"size": 20} )
#ax = sns.heatmap(report_heat, annot=True, fmt='.3f', cmap='Greens', vmin=0, vmax=1, cbar=True)
ax.set_title('Classification Report (heatmap)')
ax.set_xlabel('Metric')
ax.set_ylabel('Class')
ax.set_yticklabels(ax.get_yticklabels(), rotation=0, va='center')
plt.tight_layout()
plt.show()

# 4) (optional) Confusion matrix heatmap (you already have this)
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6), dpi=200)
ax = sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
                 xticklabels=class_names, yticklabels=class_names)
ax.set_title('Confusion Matrix')
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
plt.tight_layout()
plt.show()


In [ ]:
#Print the classification report of the tested data
pred_labels = model64.predict(X_test)
#Since the labels are softmax arrays, we need to roundoff to have it in the form of 0s and 1s,
#similar to the test_labels
def roundoff(arr):
    """To round off according to the argmax of each predicted label array. """
    arr[np.argwhere(arr != arr.max())] = 0
    arr[np.argwhere(arr == arr.max())] = 1
    return arr

for labels in pred_labels:
    labels = roundoff(labels)

# Classification report
#print(classification_report(np.argmax(y_test, axis=1), np.argmax(pred_labels, axis=1)))
print(classification_report(y_test, pred_labels, target_names=class_names, digits=4))


# Plot the confusion matrix to understand the classification in detailsnio
pred_ls = np.argmax(pred_labels, axis=1)
test_ls = np.argmax(y_test, axis=1)

conf_arr = confusion_matrix(test_ls, pred_ls)

plt.figure(figsize=(16, 12), dpi=200, facecolor='w', edgecolor='k')

#ax = sns.heatmap(conf_arr, cmap='Greens', annot=True, fmt='d', xticklabels=class_names,yticklabels=class_names, annot_kws={"fontsize": 24})
 #Plot heatmap
ax = sns.heatmap(conf_arr, cmap='Greens', annot=True, fmt='d')
# Increase font size for annotations
for text in ax.texts:
    text.set_fontsize(36)  # Adjust the font size as needed

# Increase font size for x-axis and y-axis labels
#plt.xticks(np.arange(len(class_names)), class_names, fontsize=14)
#plt.yticks(np.arange(len(class_names)), class_names, fontsize=14)

ax.set_xticklabels(class_names, rotation=45, ha='right', fontsize=24)
ax.set_yticklabels(class_names, rotation=0, va='center', fontsize=24)

plt.title(" 64x64 image size", fontsize=16)  # Title font size
plt.xlabel("Prediction", fontsize=18)  # X label font size
plt.ylabel("Actual", fontsize=18)  # Y label font size
plt.show(ax)

# **Visualization of features via LIME**

In [ ]:
n=1
model8f=tf.keras.models.load_model("/kaggle/working/final_AD4_CNNs_5cnnL_8f.keras")
predictions = model8f.predict(X_test[n:n+1])
print("true label: ",{np.argmax(y_test[n:n+1])})
print("predicted label: ",{np.argmax(predictions)})

In [ ]:
predictions

In [ ]:
# Get the shape of the original image
original_shape = X_test[n:n+1].shape

# Calculate the total number of pixels in the image
total_pixels = original_shape[1] * original_shape[2] * original_shape[3]

# Assuming the image has 3 color channels (RGB)
channels = 3

# Reshape the flattened data to 2D color image format
X_test_reshaped = X_test[n:n+1].reshape((1, original_shape[1], original_shape[2], channels))

# Print the shape of the reshaped array
print("Reshaped array shape:", X_test_reshaped.shape)

import cv2

# Convert RGB image to grayscale
gray_image = cv2.cvtColor(X_test_reshaped[0], cv2.COLOR_RGB2GRAY)

# Expand dimensions to make it 3D (2D image with a single channel)
gray_image_3d = gray_image[:, :, np.newaxis]

# Print the shape of the grayscale image
print("Grayscale image shape:", gray_image_3d.shape)
import matplotlib.pyplot as plt

# Display the grayscale image
plt.imshow(gray_image_3d[:, :, 0], cmap='gray')
plt.axis('off')  # Turn off axis
plt.show()

In [ ]:
from lime.lime_image import LimeImageExplainer 
explainer = LimeImageExplainer() 
explanation = explainer.explain_instance(gray_image_3d[:, :, 0], model8f.predict, top_labels=2, num_samples=100, random_seed=42) 
explanation

In [ ]:
n=506
predictions = model8f.predict(X_test[n:n+1])
predicted_class_index = np.argmax(predictions)

# Assuming you have a list of class labels
#class_labels = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4']

#nf=6 # number of features
# Get the label corresponding to the predicted class index
predicted_label = class_names[predicted_class_index]
print("true label: ",{np.argmax(y_test[n:n+1])})
print("predicted label_claaa: ",{np.argmax(predictions)})
print("predicted_label:",predicted_label )
class_names
predicted_class_index


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime.lime_image import LimeImageExplainer 
from skimage.segmentation import mark_boundaries

# Assuming X_test_reshaped, gray_image_3d, and explanation are already defined as per your code
# Get the shape of the original image
original_shape = X_test[n:n+1].shape

# Calculate the total number of pixels in the image
total_pixels = original_shape[1] * original_shape[2] * original_shape[3]

# Assuming the image has 3 color channels (RGB)
channels = 3

# Reshape the flattened data to 2D color image format
X_test_reshaped = X_test[n:n+1].reshape((1, original_shape[1], original_shape[2], channels))

# Print the shape of the reshaped array
print("Reshaped array shape:", X_test_reshaped.shape)

import cv2

# Convert RGB image to grayscale
gray_image = cv2.cvtColor(X_test_reshaped[0], cv2.COLOR_RGB2GRAY)

# Expand dimensions to make it 3D (2D image with a single channel)
gray_image_3d = gray_image[:, :, np.newaxis]

############LIME EXPLANER

explainer = LimeImageExplainer() 
explanation = explainer.explain_instance(gray_image_3d[:, :, 0], model8f.predict, top_labels=4, num_samples=1050, random_seed=42) 
explanation


# Display images in a row
fig, axes = plt.subplots(1, 4, figsize=(20, 5))  # Create a row of 4 images

# First image: grayscale image
axes[0].imshow(gray_image_3d[:, :, 0], cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')  # Turn off axis

nf=2

# Second image: image with mask (positive only, num_features=2)
temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=True, num_features=nf, hide_rest=True)
axes[1].imshow(mark_boundaries(temp, mask))
axes[1].set_title('Top Positive features')
axes[1].axis('off')  # Turn off axis

# Third image: image with mask (positive only, num_features=2, hide_rest=False)
temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=True, num_features=nf, hide_rest=False)
axes[2].imshow(mark_boundaries(temp, mask))
axes[2].set_title('Top Positive features (Full Image)')
axes[2].axis('off')  # Turn off axis

# Fourth image: image with mask (positive + negative, num_features=6)
temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=False, num_features=nf, hide_rest=False)
axes[3].imshow(mark_boundaries(temp, mask))
axes[3].set_title('Top Positive & Negative Features')
axes[3].axis('off')  # Turn off axis

# Display the images
plt.tight_layout()  # Adjust layout to avoid overlap
plt.show()

print("true label: ",{np.argmax(y_test[n:n+1])})

predicted_class_index = np.argmax(predictions)
# Get the label corresponding to the predicted class index
predicted_label = class_names[predicted_class_index]
print("predicted_label:",predicted_label )
print("predicted_label:",predicted_class_index)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime.lime_image import LimeImageExplainer 
from skimage.segmentation import mark_boundaries

n=444
# Get the shape of the original image
original_shape = X_test[n:n+1].shape

# Calculate the total number of pixels in the image
total_pixels = original_shape[1] * original_shape[2] * original_shape[3]

# Assuming the image has 3 color channels (RGB)
channels = 3

# Reshape the flattened data to 2D color image format
X_test_reshaped = X_test[n:n+1].reshape((1, original_shape[1], original_shape[2], channels))

# Print the shape of the reshaped array
print("Reshaped array shape:", X_test_reshaped.shape)

import cv2

# Convert RGB image to grayscale
gray_image = cv2.cvtColor(X_test_reshaped[0], cv2.COLOR_RGB2GRAY)

# Expand dimensions to make it 3D (2D image with a single channel)
gray_image_3d = gray_image[:, :, np.newaxis]

############LIME EXPLANER

explainer = LimeImageExplainer() 
explanation = explainer.explain_instance(gray_image_3d[:, :, 0], model8f.predict, top_labels=2, num_samples=500, random_seed=42) 
explanation


# Display images in a row
fig, axes = plt.subplots(1, 4, figsize=(20, 5))  # Create a row of 4 images

# First image: grayscale image
axes[0].imshow(gray_image_3d[:, :, 0], cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')  # Turn off axis

nf=2

# Second image: image with mask (positive only, num_features=2)
temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=True, num_features=nf, hide_rest=True)
axes[1].imshow(mark_boundaries(temp, mask))
axes[1].set_title('Top Positive features')
axes[1].axis('off')  # Turn off axis

# Third image: image with mask (positive only, num_features=2, hide_rest=False)
temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=True, num_features=nf, hide_rest=False)
axes[2].imshow(mark_boundaries(temp, mask))
axes[2].set_title('Top Positive features (Full Image)')
axes[2].axis('off')  # Turn off axis

# Fourth image: image with mask (positive + negative, num_features=6)
temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=False, num_features=nf, hide_rest=False)
axes[3].imshow(mark_boundaries(temp, mask))
axes[3].set_title('Top Positive & Negative Features')
axes[3].axis('off')  # Turn off axis

# Display the images
plt.tight_layout()  # Adjust layout to avoid overlap
plt.show()

print("true label: ",{np.argmax(y_test[n:n+1])})

predicted_class_index = np.argmax(predictions)
# Get the label corresponding to the predicted class index
predicted_label = class_names[predicted_class_index]
print("predicted_label:",predicted_label )
print("predicted_label:",predicted_class_index)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lime.lime_image import LimeImageExplainer 
from skimage.segmentation import mark_boundaries

n=300
# Get the shape of the original image
original_shape = X_test[n:n+1].shape

# Calculate the total number of pixels in the image
total_pixels = original_shape[1] * original_shape[2] * original_shape[3]

# Assuming the image has 3 color channels (RGB)
channels = 3

# Reshape the flattened data to 2D color image format
X_test_reshaped = X_test[n:n+1].reshape((1, original_shape[1], original_shape[2], channels))

# Print the shape of the reshaped array
print("Reshaped array shape:", X_test_reshaped.shape)

import cv2

# Convert RGB image to grayscale
gray_image = cv2.cvtColor(X_test_reshaped[0], cv2.COLOR_RGB2GRAY)

# Expand dimensions to make it 3D (2D image with a single channel)
gray_image_3d = gray_image[:, :, np.newaxis]

############LIME EXPLANER

explainer = LimeImageExplainer() 
explanation = explainer.explain_instance(gray_image_3d[:, :, 0], model8f.predict, top_labels=2, num_samples=500, random_seed=42) 
explanation


# Display images in a row
fig, axes = plt.subplots(1, 4, figsize=(20, 5))  # Create a row of 4 images

# First image: grayscale image
axes[0].imshow(gray_image_3d[:, :, 0], cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')  # Turn off axis

nf=2

# Second image: image with mask (positive only, num_features=2)
temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=True, num_features=nf, hide_rest=True)
axes[1].imshow(mark_boundaries(temp, mask))
axes[1].set_title('Top Positive features')
axes[1].axis('off')  # Turn off axis

# Third image: image with mask (positive only, num_features=2, hide_rest=False)
temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=True, num_features=nf, hide_rest=False)
axes[2].imshow(mark_boundaries(temp, mask))
axes[2].set_title('Top Positive features (Full Image)')
axes[2].axis('off')  # Turn off axis

# Fourth image: image with mask (positive + negative, num_features=6)
temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=False, num_features=nf, hide_rest=False)
axes[3].imshow(mark_boundaries(temp, mask))
axes[3].set_title('Top Positive & Negative Features')
axes[3].axis('off')  # Turn off axis

# Display the images
plt.tight_layout()  # Adjust layout to avoid overlap
plt.show()

print("true label: ",{np.argmax(y_test[n:n+1])})

predicted_class_index = np.argmax(predictions)
# Get the label corresponding to the predicted class index
predicted_label = class_names[predicted_class_index]
print("predicted_label:",predicted_label )
print("predicted_label:",predicted_class_index)

In [ ]:
img = np.squeeze(gray_image_3d)
explanation = explainer.explain_instance(img, model8f.predict, top_labels=2, num_samples=100)

In [ ]:
#predictions = model2.predict(X_test_reshaped )
#explainer = lime_image.LimeImageExplainer()
temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=True, num_features=2, hide_rest=True)
imgExplained = mark_boundaries(temp, mask)
# plot image and mask together
plt.imshow(mark_boundaries(temp  , mask))

In [ ]:


#predictions = model2.predict(X_test_reshaped )
#explainer = lime_image.LimeImageExplainer()
temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=True, num_features=2, hide_rest=False)
imgExplained = mark_boundaries(temp, mask)
# plot image and mask together
plt.imshow(mark_boundaries(temp  , mask))


In [ ]:


temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=False, num_features=4, hide_rest=False)
plt.imshow(mark_boundaries(temp, mask))



In [ ]:


temp, mask = explanation.get_image_and_mask(predicted_class_index, positive_only=False, num_features=6, hide_rest=False)
plt.imshow(mark_boundaries(temp, mask))



In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# Define the paths
dataset_path = "/kaggle/input/alzheimer-mri-4-classes-dataset/Alzheimer_MRI_4_classes_dataset"  # Replace with your actual path
classes = ["MildDemented", "ModerateDemented", "NonDemented", "VeryMildDemented"]
sample_size = 5  # Number of images to display per class

# Set up the plot grid
fig, axes = plt.subplots(len(classes), sample_size, figsize=(15, 10))
fig.suptitle("Sample Images from Each Class", fontsize=16)

# Loop through each class and display sample images
for i, class_name in enumerate(classes):
    class_path = os.path.join(dataset_path, class_name)
    images = os.listdir(class_path)[:sample_size]  # Get a few sample images
    
    for j, image_name in enumerate(images):
        img_path = os.path.join(class_path, image_name)
        img = Image.open(img_path).convert("RGB")  # Convert to RGB if grayscale
        axes[i, j].imshow(img)
        axes[i, j].axis("off")  # Hide axes
        if j == 0:
            axes[i, j].set_ylabel(class_name, fontsize=12)

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()

**OASIS Dataset testing**

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

# Define the paths
dataset_path = "/kaggle/input/imagesoasis/Data"  # Replace with your actual path
classes = ["Mild Dementia", "Moderate Dementia", "Non Demented", "Very mild Dementia"]
sample_size = 5  # Number of images to display per class

# Set up the plot grid
fig, axes = plt.subplots(len(classes), sample_size, figsize=(15, 10))
fig.suptitle("Sample Images from Each Class", fontsize=16)

# Loop through each class and display sample images
for i, class_name in enumerate(classes):
    class_path = os.path.join(dataset_path, class_name)
    images = os.listdir(class_path)[:sample_size]  # Get a few sample images
    
    for j, image_name in enumerate(images):
        img_path = os.path.join(class_path, image_name)
        img = Image.open(img_path).convert("RGB")  # Convert to RGB if grayscale
        axes[i, j].imshow(img)
        axes[i, j].axis("off")  # Hide axes
        if j == 0:
            axes[i, j].set_ylabel(class_name, fontsize=12)

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.show()


In [ ]:
model8f=tf.keras.models.load_model("/kaggle/working/final_AD4_CNNs_5cnnL_8f.keras")

In [ ]:
test_scores = model8f.evaluate(X_test, y_test)
print("Testing Accuracy: %.4f%%" % (test_scores[1] * 100))

In [ ]:
tf.keras.utils.plot_model(model8f, to_file="model8f_plot_5cnnLD.png", show_shapes=True, show_layer_names=True)